In [59]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests
import re

# Chrome 드라이버 설정
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 필요시 활성화
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Selenium 드라이버 실행
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=chrome_options
)

# 스크롤 제어 함수
def scroll_down(driver, pause_time, steps=10):
    """
    페이지를 점진적으로 스크롤하여 동적 콘텐츠 로드.
    
    :param driver: Selenium WebDriver 인스턴스
    :param pause_time: 스크롤 후 대기 시간
    :param steps: 몇 번에 걸쳐 스크롤할지
    """
    page_height = driver.execute_script("return document.body.scrollHeight")
    for step in range(steps):
        driver.execute_script(f"window.scrollTo(0, {(step + 1) * page_height / steps});")
        time.sleep(pause_time)

# 쿠팡 검색 결과 URL
url = "https://www.coupang.com/np/search?component=&q=%EB%86%8D%EC%97%85&channel=user"
driver.get(url)

# 페이지 로드 대기
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
)

# 점진적으로 스크롤
scroll_down(driver, pause_time=2, steps=20)

# 상품 리스트 로드 대기
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'li.search-product'))
)

# 저장 경로 설정
save_path = os.path.join(os.getcwd(), "이미지/기타/농업")
os.makedirs(save_path, exist_ok=True)

# 상품 정보 가져오기
products = driver.find_elements(By.CSS_SELECTOR, 'li.search-product')
valid_count = 0  # 유효한 상품 개수

for product in products:
    if valid_count >= 20:  # 최대 20개 상품만 수집
        break

    try:
        # 상품 이름 가져오기
        name_element = product.find_element(By.CSS_SELECTOR, 'div.name')
        name = name_element.text.strip()  # 공백 제거
        if not name:  # 상품 이름이 비어 있으면 건너뛰기
            continue

        # 상품 이미지 URL 가져오기
        img_element = product.find_element(By.CSS_SELECTOR, 'img.search-product-wrap-img')
        img_url = img_element.get_attribute('src')
        if not img_url or not img_url.startswith('http'):  # 이미지 URL이 없거나 유효하지 않으면 건너뛰기
            continue

        # 파일명 처리
        safe_name = re.sub(r'[\\/*?:"<>|]', "_", name)  # 특수 문자 제거

        # 이미지 다운로드
        img_data = requests.get(img_url).content
        with open(f'{save_path}/{safe_name}.jpg', 'wb') as file:
            file.write(img_data)

        # 출력 및 카운트 증가
        print(f"{valid_count + 1}. 상품 이름: {name}")
        print(f"이미지 저장 완료: {safe_name}.jpg")
        valid_count += 1

    except Exception as e:
        print(f"상품 처리 오류: {e}")

# 드라이버 종료
driver.quit()
print(f"상품 이름 및 이미지 크롤링 완료! 총 {valid_count}개의 상품을 저장했습니다.")

1. 상품 이름: 농업용 제초매트 50~300cm 공장직영 거성농자재 (마대재질), 1개
이미지 저장 완료: 농업용 제초매트 50~300cm 공장직영 거성농자재 (마대재질), 1개.jpg
2. 상품 이름: 멀칭타공기 길이조절 막대형 구멍 뚫기 천공기 펀칭 유공 모종심기 구멍파기, 5cm, 1개
이미지 저장 완료: 멀칭타공기 길이조절 막대형 구멍 뚫기 천공기 펀칭 유공 모종심기 구멍파기, 5cm, 1개.jpg
3. 상품 이름: 아리수 농업용 여과기 50mm, 1개
이미지 저장 완료: 아리수 농업용 여과기 50mm, 1개.jpg
4. 상품 이름: 앙플랜트 다용도 강철쇠삽, 1개
이미지 저장 완료: 앙플랜트 다용도 강철쇠삽, 1개.jpg
5. 상품 이름: 농업용 제초매트 50~300cm 공장직영 거성농자재 (마대재질), 1개
이미지 저장 완료: 농업용 제초매트 50~300cm 공장직영 거성농자재 (마대재질), 1개.jpg
6. 상품 이름: 농업용 부직포 제초부직포 50~300cm 및 고정핀 공장직영 거성농자재, 흑 부직포, 폭120cm 길이200m(14.4kg), 1개
이미지 저장 완료: 농업용 부직포 제초부직포 50~300cm 및 고정핀 공장직영 거성농자재, 흑 부직포, 폭120cm 길이200m(14.4kg), 1개.jpg
7. 상품 이름: 국산 텃밭용 멀칭비닐 소포장 20m 주말농장 농사용 농업용 흑색 검정 무공 비닐, 120cm(펼친폭)x20m, 1개
이미지 저장 완료: 국산 텃밭용 멀칭비닐 소포장 20m 주말농장 농사용 농업용 흑색 검정 무공 비닐, 120cm(펼친폭)x20m, 1개.jpg
8. 상품 이름: 이륜차 손수레 농업용 공업용 리어카, 이륜차(PVC), 에어바퀴, 1개
이미지 저장 완료: 이륜차 손수레 농업용 공업용 리어카, 이륜차(PVC), 에어바퀴, 1개.jpg
9. 상품 이름: 국산 농업용 멀칭 롤 비닐 150cm x 200m 0.02mm 농사용 필름 텃밭 마늘 고추, 1개
이미지 저장 완료: 국산 농업용 멀칭 롤 비닐 150cm x 200m 